Avant de passer au web scraping proprement dit, découvrons les propriétés classet id.  
Ces propriétés spéciales donnent des noms aux éléments HTML et facilitent leur interaction lors du scrapp.  
Un élément peut avoir plusieurs classes et une classe peut être partagée entre des éléments. Chaque élément ne peut avoir qu'un seul identifiant, et un identifiant ne peut être utilisé qu'une seule fois sur une page.  
Les classes et les identifiants sont facultatifs et tous les éléments n'en auront pas.

In [1]:
import requests
page = requests.get("https://www.fmd-sa.com/catalogue/1/cartons-demenagement")
page.status_code


200

In [ ]:
# Nous pouvons imprimer le contenu HTML de la page en utilisant la content propriété :
page.content


Analyser une page avec BeautifulSoup  
Nous pouvons utiliser la bibliothèque BeautifulSoup pour analyser ce document et extraire le texte de la p balise.  
Nous devons d'abord importer la bibliothèque et créer une instance de la BeautifulSoup classe pour analyser notre document :

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

Nous pouvons maintenant imprimer le contenu HTML de la page, bien formaté, en utilisant la méthode prettify sur le BeautifulSoup objet.

In [ ]:
print(soup.prettify())

Cette étape n'est pas strictement nécessaire, et nous ne nous en soucierons pas toujours, mais il peut être utile de regarder du HTML embelli pour rendre la structure des balises et où sont imbriquées plus facile à voir.  
Comme toutes les balises sont imbriquées, nous pouvons parcourir la structure d'un niveau à la fois.  
Nous pouvons d'abord sélectionner tous les éléments au niveau supérieur de la page en utilisant la children propriété de soup.

Notez que children renvoie un générateur de liste, nous devons donc appeler la list fonction dessus :

In [ ]:
list(soup.children)

Ce qui précède nous indique qu'il y a deux balises au niveau supérieur de la page  
la "!DOCTYPE html" balise initiale et la "html" balise.  
Il y a aussi un caractère de retour à la ligne ( n) dans la liste.  
Voyons quel est le type de chaque élément de la liste :

In [6]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]  
Comme nous pouvons le voir, tous les éléments sont des BeautifulSoupobjets :  

Le premier est un Doctypeobjet, qui contient des informations sur le type du document.
Le second est un NavigableString, qui représente le texte trouvé dans le document HTML.
L'élément final est un Tagobjet, qui contient d'autres balises imbriquées.
Le type d'objet le plus important, et celui dont nous traiterons le plus souvent, est l' Tagobjet.  

L' Tagobjet nous permet de naviguer dans un document HTML et d'extraire d'autres balises et du texte. Vous pouvez en savoir plus sur les différents BeautifulSoupobjets ici .

In [ ]:
# Chaque élément de la liste renvoyée par la childrenpropriété est également un BeautifulSoupobjet, nous pouvons donc également appeler la childrenméthode sur html.
# Maintenant, nous pouvons trouver les enfants à l'intérieur de la html balise :

html = list(soup.children)[2]
list(html.children)

In [ ]:
# Comme nous pouvons le voir ci-dessus, il y a deux balises ici head, et body. 
# Nous voulons extraire le texte à l'intérieur de la pbalise, nous allons donc plonger dans le corps :
body = list(html.children)[3]
body

In [ ]:
# Maintenant, nous pouvons obtenir la pbalise en trouvant les enfants de la balise body :
list(body.children)

In [ ]:
# isoler la balise p
p = list(body.children)[1]
p

In [ ]:
# Une fois que nous avons isolé la balise, nous pouvons utiliser la get_textméthode pour extraire tout le texte à l'intérieur de la balise :
p.get_text()

### Trouver toutes les instances d'une balise à la fois  
Ce que nous avons fait ci-dessus était utile pour comprendre comment naviguer dans une page, mais il fallait beaucoup de commandes pour faire quelque chose d'assez simple.  
Si nous voulons extraire une seule balise, nous pouvons plutôt utiliser la find_allméthode , qui trouvera toutes les instances d'une balise sur une page.

In [12]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')

[<p>© 2020 FMD SA - <a href="/catalogue/1/cartons-demenagement">Carton demenagement</a> - <a href="/carton-demenagement-paris.php">Paris</a> - <a href="/carton-demenagement-lyon.php">Lyon</a> - <a href="/carton-demenagement-marseille.php">Marseille</a> - <a href="/carton-demenagement-bordeaux.php">Bordeaux</a> - <a href="/conseils.php">Conseils</a> - <a href="/mentions-legales.php">Mentions légales</a> - Photos non contractuelles</p>]

Notez que find_all renvoie une liste, nous devrons donc la parcourir ou utiliser l'indexation de liste pour extraire le texte :

In [13]:
soup.find_all('p')[0].get_text()

'© 2020 FMD SA - Carton demenagement - Paris - Lyon - Marseille - Bordeaux - Conseils - Mentions légales - Photos non contractuelles'

### Recherche de balises par classe et identifiant  

Nous avons introduit les classes et les identifiants plus tôt, mais nous ne savions probablement pas pourquoi ils étaient utiles.  

Les classes et les identifiants sont utilisés par CSS pour déterminer à quels éléments HTML appliquer certains styles. Mais quand on scrape, on peut aussi les utiliser pour spécifier les éléments qu'on veut scraper.  

Pour illustrer ce principe, nous allons travailler avec la page suivante :  https://dataquestio.github.io/web-scraping-pages/ids_and_classes.html.

In [ ]:
page = requests.get("https://www.fmd-sa.com/catalogue/1/cartons-demenagement")
soup = BeautifulSoup(page.content, 'html.parser')
soup

Dans l'exemple ci-dessous, nous allons chercher n'importe quel tag qui a la classe col-sm-4:

In [ ]:
# j'ai la donnée de mes 3 rangées de cartons sur FMD

soup1 = soup.find_all(class_="col-sm-4")[1]
soup2 = soup.find_all(class_="col-sm-4")[2]
soup3 = soup.find_all(class_="col-sm-4")[3]
soup3

In [ ]:
rangee3 = soup3.find_all(class_="nav nav-pills nav-stacked")
rangee3

## Utilisation des sélecteurs CSS  
Nous pouvons également rechercher des éléments à l'aide de sélecteurs CSS .  
Ces sélecteurs permettent au langage CSS de spécifier aux développeurs des balises HTML à styliser.  

Voici quelques exemples:

    p a — trouve toutes les a balises à l'intérieur d'une p balise.
    body p a — trouve toutes les abalises à l'intérieur d'une pbalise à l'intérieur d'une bodybalise.
    html body — trouve toutes les body balises à l'intérieur d'une html balise.
    p.outer-text — trouve toutes les p balises avec une classe de outer-text.
    p#first — trouve toutes les p balises avec un identifiant de first.
    body p.outer-text — trouve toutes les p balises avec une classe outer-text à l'intérieur d'une body balise.  

Les objets  BeautifulSoup prennent en charge la recherche d'une page via des sélecteurs CSS à l'aide de la select méthode.  
Nous pouvons utiliser des sélecteurs CSS pour trouver toutes les p balises de notre page qui se trouvent à l'intérieur d'un div comme ceci :

In [ ]:
nom_carton = soup.select("div div div div div li")
print(len(nom_carton))
print(nom_carton)

#itemprop = span.find_all(itemprop="name")

Notez que la select méthode ci-dessus renvoie une liste d' BeautifulSoup objets, tout comme find et find_all.

In [ ]:
# récupérer le nom des articles

page = requests.get("https://www.fmd-sa.com/catalogue/1/cartons-demenagement")
#print(page.status_code)
soup = BeautifulSoup(page.content, 'html.parser')

listProduits= soup.find(id="listeProduits")

nav_pills = listProduits.find_all(class_="nav nav-pills nav-stacked")

link_rangee = []

for r in range(len(nav_pills)):
    rangee = nav_pills[r]
    link_rangee.append(rangee)
print(link_rangee)
print(len(link_rangee))




In [19]:
li = []
for nb_rangee in range((len(link_rangee))) :
    li.append(link_rangee[nb_rangee].find_all("li"))

print(len(li))

3


In [20]:
# count item par rangée :

item_par_rangee = []
for i in range(len(li)) :
    item_par_rangee.append(len(li[i]))

print(item_par_rangee)

[9, 9, 9]


In [21]:
# récupérer le nom de tous les articles du site

article = []

for bloc in range(len(li)) : # =3

    item_rangee = item_par_rangee[i] # =9

    for line in range(0,item_rangee) :
        article.append(li[bloc][line].get_text())

print(article)
print(len(article)) # = normalement 27


['CARTON STANDARD RENFORCÉ', 'CARTON STANDARD RENFORCÉ ++', 'CARTON SUPERDEM', 'CARTON LIVRES', 'CARTON INFORMATIQUE - Grand Modèle', 'CARTON INFORMATIQUE - Petit Modèle', 'CARTON ABAT-JOUR', 'CARTON HALOGÈNE', 'CARTON TABLEAU TÉLESCOPIQUE', 'CARTON BOUTEILLE', 'CARTON POUR KIT ISOTHERME', 'KIT ISOTHERME SANS CARTON', 'BARREL PETIT MODELE', 'BARREL GRAND MODELE', 'PACK CROISILLONS BARREL ASSIETTE', 'PACK CROISILLONS BARREL VERRE', 'PENDERIE CONFECTION', 'PENDERIE ', 'TRINGLE POUR CARTON PENDERIE', 'CONTENEUR ARCHIVES', 'KIT DE DÉPANNAGE', 'CORNIÈRE À TABLEAU', 'CORNIÈRE DE PROTECTION EN CARTON', 'CONTENEUR EXPORT - Modèle C.09', 'CONTENEUR EXPORT - Modèle GMC', 'CONTENEUR EXPORT - Modèle CXC', 'CONTENEUR EXPORT - Modèle 580']
27


In [22]:
from bs4 import BeautifulSoup
import urllib.request

parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
resp = urllib.request.urlopen("https://www.fmd-sa.com/catalogue/1/cartons-demenagement")
soup2 = BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

links = []
for link in soup2.find_all('a', href=True):
    links.append(link['href'])

print(links)


['https://www.fmd-sa.com', 'https://www.fmd-sa.com', '/catalogue/1/cartons-demenagement', '/catalogue/2/adhesif-devidoirs-films-etirables', '/catalogue/3/bulles-polyethylene-bulkraft-ecokraft', '/catalogue/4/housses-pochettes-sacs-polyethylene', '/catalogue/5/housses-de-protection', '/catalogue/6/couvertures-bracelets-caoutchouc', '/catalogue/7/emballage-papier-sangles-bricoles-ficelles', '/catalogue/9/bacs-valises-demecrin-penderies-plastique', '/catalogue/10/manutention', '/catalogue/11/garde-meubles-caisse-export', '/catalogue/12/etiquettes-materiels-divers', '/catalogue/13/imprimes', '/catalogue/14/self-stockage', '/catalogue/17/kits-demenagement', '/catalogue-vetements.php', '/FMD-Catalogue_General-2021-2022_BD.pdf', '/FMD-Catalogue_General-2021-2022_BD.pdf', '/', '/presentation.php', '#modal-cat', '/conseils.php', '/catalogue/16/location', 'https://boutique.fmd-sa.com', '/carton-line.php', '/contact.php', '/catalogue/1/cartons-demenagement', 'https://boutique.fmd-sa.com', '/catal

In [23]:
# là je n'obtiens que les premiers liens de la colonne concernant la class col sm 4
productDivs = soup.findAll('div', attrs={'class' : 'col-sm-4'})
for div in productDivs:
    print(div.find('a')['href'])

https://www.fmd-sa.com
/catalogue/1/cartons-demenagement/41/carton-demenagement
/catalogue/1/cartons-demenagement/46/carton-bouteille
/catalogue/1/cartons-demenagement/50/barre-porte-cintres


In [24]:
productDivs2 = soup.findAll('ul', attrs={'class' : 'nav nav-pills nav-stacked'})
for div in productDivs2:
    print(div.find('a')['href'])

/catalogue/1/cartons-demenagement/41/carton-demenagement
/catalogue/1/cartons-demenagement/46/carton-bouteille
/catalogue/1/cartons-demenagement/50/barre-porte-cintres


In [25]:

link_colonne = soup.findAll('div', attrs={'class' : 'col-sm-4'})
len(link_colonne) #= 4

number_link = 0

for i in range(len(link_colonne)) :

    number_link += len(link_colonne[i])

print(number_link)

# je suis censée en avoir 27 !



14


In [26]:
tableau = soup.find_all(class_='col-sm-4')
tableau1 = tableau[1] # il faut le tablaeau 1,2,3
tableau0 = tableau[0] # pas besoin
print(tableau0)


<div class="col-sm-4 col-xs-8">
<div class="h1"><a href="https://www.fmd-sa.com">FMD - Cartons, Fournitures et Matériels de Déménagement</a></div>
<a href="https://www.fmd-sa.com" title="FMD - Cartons, Fournitures et Matériels de Déménagement"><img alt="FMD - Cartons, Fournitures et Matériels de Déménagement" class="img-responsive" id="logo" src="/img/2014/logo-fmd.png"/></a>
</div>


In [27]:
Objets = []

for i in tableau1.find_all('li') :
    #print(i)
    Objets.append(i.text)

print(Objets)

['CARTON STANDARD RENFORCÉ', 'CARTON STANDARD RENFORCÉ ++', 'CARTON SUPERDEM', 'CARTON LIVRES', 'CARTON INFORMATIQUE - Grand Modèle', 'CARTON INFORMATIQUE - Petit Modèle', 'CARTON ABAT-JOUR', 'CARTON HALOGÈNE', 'CARTON TABLEAU TÉLESCOPIQUE']


In [28]:
article = []

for i in range(len(tableau)) :
    #bloc_objet = []
    for j in tableau[i].find_all('li') :
        #bloc_objet.append(j.text)
        article.append(j.text)

print(article)
print(len(article))

['CARTON STANDARD RENFORCÉ', 'CARTON STANDARD RENFORCÉ ++', 'CARTON SUPERDEM', 'CARTON LIVRES', 'CARTON INFORMATIQUE - Grand Modèle', 'CARTON INFORMATIQUE - Petit Modèle', 'CARTON ABAT-JOUR', 'CARTON HALOGÈNE', 'CARTON TABLEAU TÉLESCOPIQUE', 'CARTON BOUTEILLE', 'CARTON POUR KIT ISOTHERME', 'KIT ISOTHERME SANS CARTON', 'BARREL PETIT MODELE', 'BARREL GRAND MODELE', 'PACK CROISILLONS BARREL ASSIETTE', 'PACK CROISILLONS BARREL VERRE', 'PENDERIE CONFECTION', 'PENDERIE ', 'TRINGLE POUR CARTON PENDERIE', 'CONTENEUR ARCHIVES', 'KIT DE DÉPANNAGE', 'CORNIÈRE À TABLEAU', 'CORNIÈRE DE PROTECTION EN CARTON', 'CONTENEUR EXPORT - Modèle C.09', 'CONTENEUR EXPORT - Modèle GMC', 'CONTENEUR EXPORT - Modèle CXC', 'CONTENEUR EXPORT - Modèle 580']
27


In [29]:
url = []

tableau = soup.find_all(class_='col-sm-4')

for t in range(1,4) :

    for l in tableau[t].find_all('a'):
        url.append('https://www.fmd-sa.com/' + l.get('href'))

print(url[0])
print(len(url))

https://www.fmd-sa.com//catalogue/1/cartons-demenagement/41/carton-demenagement
27


In [50]:
print(url)

['https://www.fmd-sa.com//catalogue/1/cartons-demenagement/41/carton-demenagement', 'https://www.fmd-sa.com//catalogue/1/cartons-demenagement/42/carton-demenagement-tres-resistant', 'https://www.fmd-sa.com//catalogue/1/cartons-demenagement/43/carton-garde-meuble-export', 'https://www.fmd-sa.com//catalogue/1/cartons-demenagement/44/carton-demenagement-objets-lourds', 'https://www.fmd-sa.com//catalogue/1/cartons-demenagement/71/carton-informatique-grand-modele', 'https://www.fmd-sa.com//catalogue/1/cartons-demenagement/72/carton-informatique-petit-modele', 'https://www.fmd-sa.com//catalogue/1/cartons-demenagement/66/carton-abat-jour', 'https://www.fmd-sa.com//catalogue/1/cartons-demenagement/67/carton-halogene', 'https://www.fmd-sa.com//catalogue/1/cartons-demenagement/65/carton-tableau-telescopique', 'https://www.fmd-sa.com//catalogue/1/cartons-demenagement/46/carton-bouteille', 'https://www.fmd-sa.com//catalogue/1/cartons-demenagement/68/carton-pour-kit-isotherme', 'https://www.fmd-sa.

In [30]:
# récupération de tous les nombres dans la description de chaque article 

import re

number_in_description = []

for i in range(len(url)) :
    
    page_article = requests.get(f'{url[i]}')

    soup_article = BeautifulSoup(page_article.content, 'html.parser')

    descriptif = soup_article.find_all(class_='descriptif', itemprop='description')[0].get_text()
    
    regex = re.findall('\d+', descriptif)
    number_in_description.append(regex)


print(number_in_description)



[['550', '350', '300', '20'], ['1', '1'], ['550', '350', '330', '40'], ['350', '275', '300', '20'], ['550', '465', '610', '40'], ['525', '425', '470', '40'], ['700', '700', '500', '30'], ['2000', '350', '350', '40'], ['1000', '150', '900', '40'], ['339', '253', '330', '20', '12', '85', '85', '330', '12'], ['350', '350', '465', '24'], [], ['450', '450', '565', '75'], ['450', '450', '735', '100'], ['24', '1', '2'], ['100', '534', '3', '75', '543', '2'], ['500', '500', '1200', '40'], ['500', '500', '820', '40', '09', '580'], ['500'], ['530', '360', '270', '20'], ['30', '2'], ['820', '90', '565', '30'], ['75', '75', '200', '100'], ['1190', '990', '830', '0', '98', '3', '500', '9'], ['15', '1520', '1220', '1529', '1447', '1123', '1390', '2', '20', '3', '38'], ['15', '2100', '1375', '1554', '2023', '1270', '1415', '3', '63', '3', '55'], ['15', '2210', '1185', '2210', '2154', '1107', '2042', '4', '87', '3', '82']]


In [31]:
rangee1 = number_in_description[0:9]
rangee2 = number_in_description[9:18]
rangee3 = number_in_description[18:27]


In [32]:
rangee1_test = rangee1.copy()
print(rangee1_test)

rangee2_test = rangee2.copy()
print(rangee2_test)

rangee3_test = rangee3.copy()
print(rangee3_test)

[['550', '350', '300', '20'], ['1', '1'], ['550', '350', '330', '40'], ['350', '275', '300', '20'], ['550', '465', '610', '40'], ['525', '425', '470', '40'], ['700', '700', '500', '30'], ['2000', '350', '350', '40'], ['1000', '150', '900', '40']]
[['339', '253', '330', '20', '12', '85', '85', '330', '12'], ['350', '350', '465', '24'], [], ['450', '450', '565', '75'], ['450', '450', '735', '100'], ['24', '1', '2'], ['100', '534', '3', '75', '543', '2'], ['500', '500', '1200', '40'], ['500', '500', '820', '40', '09', '580']]
[['500'], ['530', '360', '270', '20'], ['30', '2'], ['820', '90', '565', '30'], ['75', '75', '200', '100'], ['1190', '990', '830', '0', '98', '3', '500', '9'], ['15', '1520', '1220', '1529', '1447', '1123', '1390', '2', '20', '3', '38'], ['15', '2100', '1375', '1554', '2023', '1270', '1415', '3', '63', '3', '55'], ['15', '2210', '1185', '2210', '2154', '1107', '2042', '4', '87', '3', '82']]


In [33]:
for i in range(len(rangee1_test)) :
    print(len(rangee1_test[i]))

4
2
4
4
4
4
4
4
4


In [34]:
for i in range(len(rangee1_test)) :
    if (len(rangee1_test[i])) < 3 :
        rangee1_test[i] = rangee1_test[i-1]
    elif (len(rangee2_test[i])) < 3 :
        rangee2_test[i] = rangee2_test[i-1]
print(rangee1_test)
print(rangee2_test)
    



[['550', '350', '300', '20'], ['550', '350', '300', '20'], ['550', '350', '330', '40'], ['350', '275', '300', '20'], ['550', '465', '610', '40'], ['525', '425', '470', '40'], ['700', '700', '500', '30'], ['2000', '350', '350', '40'], ['1000', '150', '900', '40']]
[['339', '253', '330', '20', '12', '85', '85', '330', '12'], ['350', '350', '465', '24'], ['350', '350', '465', '24'], ['450', '450', '565', '75'], ['450', '450', '735', '100'], ['24', '1', '2'], ['100', '534', '3', '75', '543', '2'], ['500', '500', '1200', '40'], ['500', '500', '820', '40', '09', '580']]


In [35]:
# récupérer les dimensions largeur X hauteur x prof pour la rangée 1/3

for iteration in range(len(rangee1_test)) : # 9 itérations

    for number in range(len(rangee1_test[iteration])): 
        if len(rangee1_test[iteration]) == 4 :
            rangee1_test[iteration].remove(rangee1_test[iteration][3])

        elif len(rangee1_test[iteration]) < 4 or len(rangee1_test[iteration]) > 4 :   
            pass      


print(rangee1_test)

[['550', '350', '300'], ['550', '350', '300'], ['550', '350', '330'], ['350', '275', '300'], ['550', '465', '610'], ['525', '425', '470'], ['700', '700', '500'], ['2000', '350', '350'], ['1000', '150', '900']]


In [36]:
for iteration in range(len(rangee2_test)) : # 9 itérations

    for number in range(len(rangee1_test[iteration])): 
        
        if len(rangee1_test[iteration]) == 4 :
            rangee1_test[iteration].remove(rangee1_test[iteration][3])

        elif len(rangee1_test[iteration]) < 4 or len(rangee1_test[iteration]) > 4 :   
            pass      


print(rangee1_test)

[['550', '350', '300'], ['550', '350', '300'], ['550', '350', '330'], ['350', '275', '300'], ['550', '465', '610'], ['525', '425', '470'], ['700', '700', '500'], ['2000', '350', '350'], ['1000', '150', '900']]


In [37]:
test = rangee2_test.copy()

In [38]:
if len(test[-1]) > 4 :
    print(test[-1])
    print(test[-1][0:3])
else :
    print('non concerné')

['500', '500', '820', '40', '09', '580']
['500', '500', '820']


In [39]:
if len(test[0]) > 4 :
    print(test[0])
    print(test[0][0:3])
else :
    print('non concerné')

['339', '253', '330', '20', '12', '85', '85', '330', '12']
['339', '253', '330']


In [40]:
# dimensions, une fois dans l'url
# Dimension ext. : 2210 x 1185 x 2210 mm largeur X hauteur x prof
# Dimension int. : 2154 x 1107 x 2042 mm
# Volume utile : 4,87 m3
# Tare : 82 Kg

#longueur_tringle = rangee3_test[0]

#rangee1_2 = rangee1_test + rangee2_test
#print(len(rangee1_2))

for iteration in range(len(test)) : # 9 itérations

    if len(test[iteration]) > 4 : 
        print(test[iteration])
        test[iteration] == test[iteration][0:3]
    
    elif len(test[iteration]) == 4 :
        print(test[iteration])
        test[iteration].remove(test[iteration][3])

    elif len(test[iteration]) < 4  :
        print(test[iteration])   
        pass   


#print(test)
print(len(test[0]))



['339', '253', '330', '20', '12', '85', '85', '330', '12']
['350', '350', '465', '24']
['350', '350', '465']
['450', '450', '565', '75']
['450', '450', '735', '100']
['24', '1', '2']
['100', '534', '3', '75', '543', '2']
['500', '500', '1200', '40']
['500', '500', '820', '40', '09', '580']
9


In [41]:
for iteration in range(len(test)) : # 9 itérations

    if len(test[iteration]) > 4 :
        print(f"C'est la liste suivante : {test[iteration]}")
        print(f"Il y a {len(test[iteration])} itérations")
        print(f"c'est l'itération n° {iteration}")
        test[iteration] == test[iteration][0:3]





C'est la liste suivante : ['339', '253', '330', '20', '12', '85', '85', '330', '12']
Il y a 9 itérations
c'est l'itération n° 0
C'est la liste suivante : ['100', '534', '3', '75', '543', '2']
Il y a 6 itérations
c'est l'itération n° 6
C'est la liste suivante : ['500', '500', '820', '40', '09', '580']
Il y a 6 itérations
c'est l'itération n° 8


In [42]:
test[0] == test[0].replace(test[0], test[0][0:3]) 
print(test[0])

AttributeError: 'list' object has no attribute 'replace'

In [ ]:
print(rangee1_2[15])
print(len(rangee1_2[15])-3)
rangee1_2[15].remove(rangee1_2[15][-3])
print(rangee1_2[15])

['100', '534', '3', '543', '2']
2
['100', '534', '543', '2']


In [ ]:
for i in range(len(rangee1_2)) :
    print(len(rangee1_2[i]))

3
3
3
3
3
3
3
3
3
9
9
9
4
4
3
6
4
6


In [ ]:
print(rangee1_2[10])
print(rangee1_2[10][3:-1])


['339', '253', '330', '20', '12', '85', '85', '330', '12']
['20', '12', '85', '85', '330']


## Combiner nos données dans une Dataframe Pandas  
Nous pouvons maintenant combiner les données dans un Pandas DataFrame et les analyser. Un DataFrame est un objet qui peut stocker des données tabulaires, ce qui facilite l'analyse des données. Si vous voulez en savoir plus sur les pandas, consultez notre cours gratuit pour commencer ici .
Pour ce faire, nous appellerons la classe DataFrame et transmettrons chaque liste d'éléments que nous avons. Nous les transmettons dans le cadre d'un dictionnaire.
Chaque clé de dictionnaire deviendra une colonne dans le DataFrame, et chaque liste deviendra les valeurs de la colonne :

In [ ]:
import pandas as pd

produits_FMD = pd.DataFrame({
    "articles": article,
    "url" : url,
    "largeur" : Largeur
})
produits_FMD

ValueError: All arrays must be of the same length